In [0]:
import pandas as pd
import numpy as np
import os

from datetime import date, datetime

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

month_day_fmt = mdates.DateFormatter("%b %d")

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

import plotly.express as px
import plotly
import plotly.tools as tls
import plotly.graph_objs as go


In [0]:
change = pd.read_csv('DATA/change.csv')
colours = ['#44546a', '#a5a5a5', '#2baae9', '#00375e', '#ff575d']
fig = px.bar(change, x='var', 
             y='predicted', facet_col='x', color='var',
             title="Air pollution in Tbilisi (percent change from pre-Covid baseline, modelled estimate)",
             color_discrete_sequence=colours)

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
fig.for_each_trace(lambda t: t.update(name=t.name.split("=")[1]))
title = 'Data: air.gov.ge, power.larc.nasa.gov (January 1, 2018 to March 25, 2018) | Deseasoned modelled estimates | CRRC-Georgia'
for n, a in enumerate(fig['layout']):
    if 'yaxis' in a:
        fig['layout'][a].update(range=[-51,55], autorange=False)
    if 'xaxis' in a:
        fig['layout'][a].update(showticklabels=False, title='')

fig.layout.yaxis.update(title='Percent change')
fig.layout.title.font.update(size=24)
fig.layout.title.update(y=0.97)
title = 'Data: air.gov.ge, power.larc.nasa.gov (January 1, 2018 to March 25, 2018) | Deseasoned modelled estimates | CRRC-Georgia'
fig.layout.xaxis2.title.update(text=title)

plotly.offline.iplot(fig, filename = 'percent_change.html')


In [0]:
df = pd.read_feather(
    "DATA/final.feather",
    columns=["ds", "CO", "NO2", "PM10", "PM25", "SO2", "cut"],
)
subs = ["CO", "NO2", "PM10", "PM25", "SO2"]

for sub in subs:
    df[sub + "_norm"] = scaler.fit_transform(df[sub].to_numpy().reshape(-1, 1))

df["pol"] = df[[i for i in df if "_norm" in i]].mean(axis=1)

overlay = df
overlay["year_day"] = overlay["ds"].dt.dayofyear
overlay["year"] = overlay["ds"].dt.year
overlay["date"] = pd.to_datetime(
    overlay["ds"].dt.strftime("2020-%m-%d"), dayfirst=False
)

day_now = datetime.now().timetuple().tm_yday
overlay = overlay[
    (overlay["year_day"] <= day_now) & (overlay["year_day"] >= 31)
]
overlay2 = (
    overlay.groupby("year").rolling("2d", on="date")["pol"].mean().reset_index()
)
overlay2 = overlay2.merge(overlay[["date", "cut"]]).drop_duplicates(subset=['date', 'year'])
overlay2["alpha"] = np.where(overlay2.year == 2020, 1.0, 0.5)


In [0]:
fig = px.line(overlay2.sort_values('date'), x='date', y='pol', color='year', color_discrete_sequence=['#a5a5a5', '#2baae9', '#00375e'])

fig.for_each_trace(lambda t: t.update(name=t.name.split("=")[1]))

fig.layout.yaxis.update(title='Pollution levels')
fig.layout.title.font.update(size=24)
fig.layout.title.update(text='Air Pollution in Tbilisi: February to March, 2018-20', y=0.97)

dts = [date(2020, 2, 26), date(2020, 3, 16), date(2020, 3, 21)]
lbs = [
    "  First case <br>  Febuary 26",
    "  Closures <br>  March 16",
    "  Emergency <br>  March 21",
]
for dt in dts:
  fig.add_shape(
          # Line Vertical
          dict(
              type="line",
              x0=dt,
              y0=0,
              x1=dt,
              y1=0.37,
              line=dict(
                  color=colours[0],
                  width=1,
                  dash="dot"
              )
  ))
  
fig.add_trace(go.Scatter(
    x=dts,
    y=[0.4,0.4,0.4],
    text=lbs,
    mode="text", showlegend=False
))

fig.layout.yaxis.update(range=[0, 0.5], autorange=False)
title = 'Data: air.gov.ge (February-March, 2018-20) | Mean normalized levels, all pollutants | CRRC-Georgia'
fig.layout.xaxis.title.update(text=title)
plotly.offline.iplot(fig, filename = 'time_plot.html')
